In [23]:
import os
import pandas as pd
import sys, pathlib, pymupdf
import chromadb
import os
import torch
import tensorflow as tf
import PyPDF2
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from transformers import pipeline
from io import BytesIO
from sentence_transformers import SentenceTransformer
#from app import from_pdf_to_text
import glob 
import os
import fitz  # PyMuPDF for PDF handling
import mobi
import docx
from chromadb.config import Settings

In [24]:
load_dotenv()

HUGGINGFACEHUB_API_TOKEN = os.getenv("HUGGINGFACEHUB_API_KEY")

In [39]:
CHROMA_DATA_PATH = r"./embeddings_database"
DATA_DIR = r"./data"
COLLECTION_NAME = "AI_related_documents"
EMBED_MODEL = "google-bert/bert-base-uncased"

EMBED_FUNCTION = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=EMBED_MODEL)

client = chromadb.PersistentClient(path = CHROMA_DATA_PATH)
collection = client.get_or_create_collection(name=COLLECTION_NAME,embedding_function = EMBED_FUNCTION,metadata={"hnsw:space": "cosine"},)


In [27]:
from transformers import BertTokenizer, BertModel  


bert_tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")
bert_model = BertModel.from_pretrained("google-bert/bert-base-uncased")

In [28]:

def get_bert_embedding(text):
    """Generates an embedding for the given text using BERT."""
    encoded_text = bert_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad(): 
        outputs = bert_model(**encoded_text)
    embeddings = outputs.last_hidden_state[:, 0, :]  
    embeddings_list = embeddings.squeeze(0).tolist() 
    return embeddings_list

In [29]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

def chunk_text(text, chunk_size=500):
    words = text.split()
    chunks = [' '.join(words[i:i + chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

dataset_dir = "C:/Users/brahe/LLM_QA/Books/ai_data"
datasets = [os.path.join(dataset_dir, f) for f in os.listdir(dataset_dir) if f.endswith('.pdf')]

print(datasets)

['C:/Users/brahe/LLM_QA/Books/ai_data\\AI-native_Memory_A_Pathway_from_LLMs_Towards_AGI.pdf', 'C:/Users/brahe/LLM_QA/Books/ai_data\\Attention_Is_All_You_Need_But_You_Dont_Need_All_O.pdf', 'C:/Users/brahe/LLM_QA/Books/ai_data\\Comparing_sparse_Llama_3_and_Llama_2_models_for_on.pdf', 'C:/Users/brahe/LLM_QA/Books/ai_data\\DeepResidualLearningforImageandVideoRecognition.pdf', 'C:/Users/brahe/LLM_QA/Books/ai_data\\Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow_ Concepts, Tools, and Techniques to Build Intelligent Systems-O’Reilly Media.pdf', "C:/Users/brahe/LLM_QA/Books/ai_data\\Jay  Alammar, Maarten Grootendorst - Hands-On Large Language Models_ Language Understanding and Generation-O'Reilly Media (2024).pdf", "C:/Users/brahe/LLM_QA/Books/ai_data\\Lewis Tunstall, Leandro von Werra, Thomas Wolf - Natural Language Processing with Transformers-O'Reilly Media (2022).pdf", 'C:/Users/brahe/LLM_QA/Books/ai_data\\Natural language processing for online applications_ text retriev

In [36]:
for pdf_path in datasets:
    pdf_text = extract_text_from_pdf(pdf_path)    
    text_chunks = chunk_text(pdf_text)    
    i = 0
    for chunk in text_chunks:
        i += 1
        chunk_id = os.path.basename(pdf_path) + f"_{i}"
        embeddings =get_bert_embedding(chunk)        
        metadata = {"file_name": os.path.basename(pdf_path) + f"_{i}"}  # Append i as string
        collection.add(
            documents=[chunk],    # Single document chunk
            embeddings=embeddings, # Single embedding for this chunk
            metadatas=[metadata] ,  # Metadata for the chunk
            ids=[chunk_id] 
        )


Exception occurred invoking consumer for subscription 61c101df93fa46a18dd2c63d673252fdto topic persistent://default/default/cb128511-1852-40cd-a068-633484703b34 'utf-8' codec can't encode character '\ud835' in position 1236: surrogates not allowed
Exception occurred invoking consumer for subscription 61c101df93fa46a18dd2c63d673252fdto topic persistent://default/default/cb128511-1852-40cd-a068-633484703b34 'utf-8' codec can't encode character '\ud835' in position 2383: surrogates not allowed
Exception occurred invoking consumer for subscription 61c101df93fa46a18dd2c63d673252fdto topic persistent://default/default/cb128511-1852-40cd-a068-633484703b34 'utf-8' codec can't encode character '\ud835' in position 474: surrogates not allowed
Exception occurred invoking consumer for subscription 61c101df93fa46a18dd2c63d673252fdto topic persistent://default/default/cb128511-1852-40cd-a068-633484703b34 'utf-8' codec can't encode character '\ud835' in position 1607: surrogates not allowed
Exception

In [37]:
collection.count()


1564

In [42]:
def perform_similarity_search(query, top_k=5):
    query_embedding = EMBED_MODEL.encode(query)  
    results = collection.query(
        query_embeddings=[query_embedding], 
        n_results=top_k
    )
    
    return results

query = "Attention mechanism"
results = perform_similarity_search(query)

# Output search results
for result in results["documents"]:
    print("Matching Document:", result)


LookupError: unknown encoding: Attention mechanism